In [1]:
%run init.ipynb

matchzoo version 1.0
`classification_task` initialized with metrics [accuracy]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.Bert.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing length_right with len: 100%|██████████| 2300/2300 [00:00<00:00, 804109.29it/s]


In [4]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='point'
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    mode='point'
)

In [5]:
padding_callback = mz.models.Bert.get_default_padding_callback()
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    callback=padding_callback
)

In [6]:
model = mz.models.Bert()

model.params['task'] = classification_task
model.params['mode'] = 'bert-base-uncased'
model.params['dropout_rate'] = 0.2

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

Bert(
  (bert): BertModule(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (dr

In [7]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 5e-5},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

from pytorch_transformers import AdamW, WarmupLinearSchedule

optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, betas=(0.9, 0.98), eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=6, t_total=-1)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [8]:
trainer.run()

[Iter-1018 Loss-0.786]:
  Validation: accuracy: 0.1551



[Iter-2036 Loss-0.370]:
  Validation: accuracy: 0.8791



[Iter-3054 Loss-0.364]:
  Validation: accuracy: 0.8791



[Iter-4072 Loss-0.364]:
  Validation: accuracy: 0.8791



[Iter-5090 Loss-0.364]:
  Validation: accuracy: 0.8791



[Iter-6108 Loss-0.364]:
  Validation: accuracy: 0.8791



[Iter-7126 Loss-0.364]:
  Validation: accuracy: 0.8791



[Iter-8144 Loss-0.364]:
  Validation: accuracy: 0.8791



[Iter-9162 Loss-0.364]:
  Validation: accuracy: 0.8791



[Iter-10180 Loss-0.364]:
  Validation: accuracy: 0.8791

Cost time: 25427.485457658768s
